In [2]:
import json
import os
from glob import glob
import pandas as pd
import re
import plotly.graph_objs as go
import scipy.stats as st
import numpy as np

# Utility Functions to read saved experimental statistics and plotting

In [3]:
def read_experiment_stats(regex):
    all_files = glob("./stats/*.json")
    files = []
    for f in all_files:
        temp = os.path.split(f)[-1]
        if re.search(regex, temp) is not None:
            files.append(f)
    # print(files)
    method_acc = []
    method_total_flops = []
    method_client_flops = []
    method_comm = []
    steps = []
    names = []

    # hparams = {}
    # for k in json.load(open(files[0], "r"))['hparams'].keys():
    #     hparams[k] = []
        

    for f in files:
        j = json.load(open(f, "r"))
        method_acc.append(j['Method Accuracy'])
        method_total_flops.append(j['Method Flops'])
        method_client_flops.append(j['Method Client Flops'])
        method_comm.append(j['Method Comm Cost'])
        steps.append(j['Steps'])
        names.append(os.path.split(f)[-1].replace(".json", ""))
        
        # for k, v in j['hparams'].items():
        #     hparams[k].append(v)

    df = pd.DataFrame(
        zip(names, method_acc, method_comm, method_total_flops, method_client_flops), 
        columns=["Experiment Name", "Method Accuracy", "Method Comm Cost", "Total Flops", "Client Flops"]
    )
    
    # Make main results table
    table = pd.DataFrame()
    table['Experiment Name'] = df['Experiment Name']
    table['Method Accuracy'] = df['Method Accuracy'].apply(lambda x: max(eval(x)))
    table['Client TFLOPS'] = df['Client Flops'].apply(lambda x: eval(x)[-1] * 1e-12)
    table['Total TFLOPS'] = df['Total Flops'].apply(lambda x: eval(x)[-1] * 1e-12)
    table['Comm Cost (GBs)'] = df['Method Comm Cost'].apply(lambda x: eval(x)[-1] * 1e-3)

    # Plot Accuracy Figure
    fig = go.Figure()
    for i in range(len(df)):
        fig.add_trace(
            go.Scatter(x=eval(steps[i]), y=eval(df['Method Accuracy'][i]), name=df['Experiment Name'][i])
        )
    fig.update_layout(title_text="",
            paper_bgcolor='rgba(255,255,255,1)',\
        plot_bgcolor='rgba(255,255,255,1)',)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,ticks='outside', showgrid=True, gridcolor="LightGray")
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridcolor="LightGray")

    fig.update_layout(template=None, xaxis_title="Training Steps", yaxis_title="Per-Client Test Accuracy", font=dict(family='Times New Roman', size=14, color='Black'))

    return df, table, fig


# SplitCIFAR (non-i.i.d) Results

In [4]:
regex = r'cifar\_NIID'

df, table, fig = read_experiment_stats(regex)
table

,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)


In [5]:
fig.show()

# CIFAR10 (i.i.d) Results

In [6]:
df, table, fig = read_experiment_stats(r'cifar\_(?!NIID)')
table

,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)


In [7]:
fig.show()

# Interrupt Duration Ablation on SplitCIFAR
- All settings are the same as CESL-6/10 except for Interrupt duration.

In [8]:
df, table, fig = read_experiment_stats(r'interrupt\_range\_expt')
table

,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)


In [9]:
fig.show()

# Interrupt Duration Ablation on CIFAR10 (i.i.d)
- All settings are the same as CESL-6/10 except for Interrupt duration.

In [10]:
df, table, fig = read_experiment_stats(r'interrupt\_range\_iid')
table

,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)


In [11]:
fig.show()

In [12]:
df, table, fig = read_experiment_stats(r'SplitFed')
table

,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)


In [13]:
fig.show()

In [14]:
df, table, fig = read_experiment_stats(r'non_iid_50')
table


,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)
0,non_iid_50_Only-Masked_45,0.885454,3.760128,26.685648,80.216064
1,non_iid_50_CESL-3-5_15,0.881319,5.384503,9.511097,14.438892
2,non_iid_50_CESL-Random-3-5_45,0.877510,5.384503,9.511097,14.438892
3,non_iid_50_Only-Masked_35,0.885065,3.760128,26.685648,80.216064
4,non_iid_50_Only-Masked_15,0.884141,3.760128,26.685648,80.216064
5,non_iid_50_CESL-Random-3-5_15,0.874569,5.384503,9.511097,14.438892
6,non_iid_50_CESL-Random-3-5_35,0.876749,5.384503,9.511097,14.438892
7,non_iid_50_CESL-3-5_45,0.874385,5.384503,9.511097,14.438892
8,non_iid_50_CESL-3-5_35,0.884008,5.384503,9.511097,14.438892
9,non_iid_50_Only-Masked_05,0.884002,3.760128,26.685648,80.216064


In [15]:
fig.show()